In [ ]:
import open3d
import scipy.io
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

# Load camera matrices
data = scipy.io.loadmat("matrices.mat")
data = data["P"]
projections = [data[0, i] for i in range(data.shape[1])]

files = sorted(glob.glob("images3/*.jpg"))
images = []
for f in files:
    im = cv2.imread(f, cv2.IMREAD_UNCHANGED).astype(float)
    im /= 255
    images.append(im[:, :, ::-1])
    

# get silouhette from images
imgH, imgW, __ = images[0].shape
silhouette = []
for im in images:
    temp = np.abs(im - [0.0, 0.0, 0.75])
    temp = np.sum(temp, axis=2)
    y, x = np.where(temp <= 1.1)
    im[y, x, :] = [0.0, 0.0, 0.0]
    im = im[:, :, 0]
    im[im > 0] = 1.0
    im = im.astype(np.uint8)

    kernel = np.ones((5, 5), np.uint8)
    im = cv2.morphologyEx(im, cv2.MORPH_OPEN, kernel)
    silhouette.append(im)

#    plt.figure()
#    plt.imshow(im)
    
#%%
# create voxel grid
s = 120
x, y, z = np.mgrid[:s, :s, :s]
pts = np.vstack((x.flatten(), y.flatten(), z.flatten())).astype(float)
pts = pts.T
nb_points_init = pts.shape[0]
xmax, ymax, zmax = np.max(pts, axis=0)
pts[:, 0] /= xmax
pts[:, 1] /= ymax
pts[:, 2] /= zmax
center = pts.mean(axis=0)
pts -= center
pts /= 5
pts[:, 2] -= 0.62

pts = np.vstack((pts.T, np.ones((1, nb_points_init))))

filled = []
for P, im in zip(projections, silhouette):
    uvs = P @ pts
    uvs /= uvs[2, :]
    uvs = np.round(uvs).astype(int)
    x_good = np.logical_and(uvs[0, :] >= 0, uvs[0, :] < imgW)
    y_good = np.logical_and(uvs[1, :] >= 0, uvs[1, :] < imgH)
    good = np.logical_and(x_good, y_good)
    indices = np.where(good)[0]
    fill = np.zeros(uvs.shape[1])
    sub_uvs = uvs[:2, indices]
    res = im[sub_uvs[1, :], sub_uvs[0, :]]
    fill[indices] = res 
    
    filled.append(fill)
filled = np.vstack(filled)

# the occupancy is computed as the number of camera in which the point "seems" not empty
occupancy = np.sum(filled, axis=0)

# Select occupied voxels
with open('files/out1.txt') as f:
    content = f.readlines()
with open('files/out.txt') as f:
    content1 = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
l=[]
content = [x.strip() for x in content] 
content1 = [x.strip() for x in content1] 
for i in range(len(content1)):
    l.append(tuple([float(content1[i].split(maxsplit=8)[0]),float(content1[i].split(maxsplit=8)[1]),float(content1[i].split(maxsplit=8)[2])]))
for i in range(len(content)):
    l.append(tuple([float(content[i].split(maxsplit=8)[0]),float(content[i].split(maxsplit=8)[1]),20-float(content[i].split(maxsplit=8)[2])]))
with open("out.obj", "w") as fout:
        for p in l:
            fout.write("v " + " ".join(map(str, p)) + "\n")
out=np.asarray(l)
point_cloud = open3d.geometry.PointCloud()
point_cloud.points = open3d.utility.Vector3dVector(out)
open3d.visualization.draw_geometries([point_cloud])